In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
import itertools
from setuptools.namespaces import flatten
from scipy import stats
import GetOldTweets3 as got


Bad key "text.kerning_factor" on line 4 in
C:\Users\User\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [4]:
# function created to form the dataset
def get_tweets(tag, state, startdate, enddate, maxtweet):
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(tag)\
                                            .setSince(startdate)\
                                            .setUntil(enddate)\
                                            .setNear(state)\
                                            .setWithin("400mi")\
                                            .setMaxTweets(maxtweet)
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    
    text_tweets = [[tw.username,
                tw.text,
                tw.date,
                tw.retweets,
                tw.favorites,
                tw.mentions,
                tw.hashtags,
                tw.geo] for tw in tweet]
    df_state= pd.DataFrame(text_tweets, columns = ['User', 'Text', 'Date', 'Favorites', 'Retweets', 'Mentions','Hashtags', 'Geolocation'])
    
    return df_state

In [5]:
#abc= get_tweets(tag='Bucaram' ,state='-1.83, -78.18', startdate="2020-04-01", enddate="2020-07-30",maxtweet= 5000)

In [6]:
#abc.loc[abc.Mentions!=''] FueraCorrupcionYA
#abc

In [2]:
#pd.Series(np.array(''.join(list(abc.Hashtags[abc.Hashtags!= '']) ).replace(' ','').split('#'))[ 
#np.array(''.join(list(abc.Hashtags[abc.Hashtags!= '']) ).replace(' ','').split('#'))!='']).value_counts().head(15)

In [16]:
# keywords related to IESS corruption in ecuador
for clave in ['daniel salcedo', 'Abraham Muñoz', 'iess', 'carnet discapacitado','Reparto Hospitales',
             '#ElPeorGobiernoDeLaHistoria','Bucaram', 'corrupcion ecuador', '#EcuadorEnCrisis',"daniel mendoza"]:
    lista_df= []
    df_tag= get_tweets(tag=clave ,state='-1.83, -78.18', startdate="2020-04-01", enddate="2020-07-15",maxtweet= 5000)
    lista_df.append(df_tag)
    df_final= pd.concat(lista_df)
    
    

In [17]:
# tweets with mentioned people
df_solo_ment= df_final[df_final.Mentions!= ''].reset_index()
df_solo_ment.User= df_solo_ment.User.str.lower()
df_solo_ment.Mentions= df_solo_ment.Mentions.str.lower()


In [18]:
# dictionary data processing for gephi:
user_unico= list(set(df_solo_ment.User))
dic_menc= dict()
for usuario in user_unico:
    lista_quienes_mencione= np.array(''.join( list( df_solo_ment.loc[df_solo_ment.User== usuario,'Mentions'] ) ).replace(' ','').split('@'))[
    np.array(''.join( list( df_solo_ment.loc[df_solo_ment.User== usuario,'Mentions'] ) ).replace(' ','').split('@'))!=''].tolist()
    mencione_df= pd.DataFrame(pd.Series(lista_quienes_mencione).value_counts()).reset_index()
    mencione_df= mencione_df.to_records(index=False)
    lista_tupla= list(mencione_df)
    dic_menc[usuario]= lista_tupla
    

In [19]:
# data processing for gephi: origin, arrival(mentioned people), weight( times mentioned)
listaOrigen=[]
listaLlegada=[]
listaArcos=[]
for idUser in dic_menc:
    listaOrigen.extend([idUser]*len(dic_menc[idUser]))
    listaLlegada.extend( list( dict(dic_menc[idUser]).keys()) )
    listaArcos.extend( list( dict(dic_menc[idUser]).values()) )
    

In [20]:
# data processing for gephi
data = pd.DataFrame(np.array(listaOrigen))
data.rename(columns={0:'Source'},inplace=True)
data['Target']= np.array(listaLlegada)
data['Weight']= np.array(listaArcos)
data['Label']= np.array(['conexion']*len(listaArcos))
data['Type']= np.array(['directed']*len(listaArcos))

In [21]:
# df for users
nod = pd.DataFrame(np.array( list(dic_menc.keys()) ))
nod.rename(columns={0:'ID'},inplace=True)
nod['Label']= nod['ID']
#nod.to_excel('nod.xlsx')

In [22]:
# export dataset for gephi
#data.to_excel('sna_IESS.xlsx')